# Data generation and simulation subsystem.
One of the most important challenge in the big data projects is finding and collecting a good quality data from a reliable source. This project is not an exception. Due to the project statement, we must work with Samsung battery usage dataset which was not publicly accessible anymore (it was publicly shared last year for a challenge). Unfortunately, as the Samsung privacy policies applies to all Samsung devices they couldn’t share this data to us and the process would take at least two months to get the required licenses. Hence, we decided to generate data. 

Having a meeting with Bandeep Singh, Data Scientist from Samsung Digital Canada, we understood their requirement and the real data specifications. So, we developed the Data Generator based on the latest standard as described in the following.

In [1]:
import random
from random import randrange
import configparser
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
import csv
import itertools
import sys

 ### This method is publicly used as convertor a number of month to its string representation.

In [2]:
def month_converter(month):
    '''
    maps month to index (string -> int)
    '''
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    return months.index(month) + 1

### Lets define some function to save the json format file as csv file
* extract_header: receive a list of json objects  and extract the header to save as csv file header (first row)
* save_json_to_csv: receive a list of json objects  and save it as the given name with csv format. it uses the extract_header.
* append_json_to_csv: append list of json objects to an existing csv file. It will create the file if it does not exist.

In [3]:

def extract_header(json_list):
    '''
    return header file
    '''
    csv_header = [ x for row in json_list for x in row.keys() ]
    csv_header = list( set( csv_header ) )
    return csv_header



def append_json_to_csv(json_list,csv_folder, csv_name, csv_header =[]):
    '''
    append list of json objects to an existing csv file. 
    it will create the file if it does not exist
    '''
    if (csv_header ==[]):
        csv_header = extract_header(json_list)
    if not(os.path.isdir(csv_folder)):
        os.makedirs(csv_folder)
        print('directory {} created to save the generated datasets'.format(csv_folder))
    print('\n file {} saving process started'.format(csv_name))# 'with the header{}'.format(csv_header))
    if  not( os.path.isfile(csv_name)):
        with open( csv_name, 'w' , newline='') as out_file:
            csv_w = csv.writer( out_file )
            csv_w.writerow( csv_header )
    with open( csv_name, 'a' , newline='') as out_file:
        csv_w = csv.writer( out_file )
        for i_r in json_list:
            csv_w.writerow( map( lambda x: i_r.get( x, "" ), csv_header ) )
    print('finished\n')


### Lets define two enum to handel 
* Battery Charging state
* User battery usage

In [4]:
def enum(**enums):
    return type('Enum', (), enums)
 
Charge_Stat = enum(CHARGING = 1, DISCHARGING = 2)
User_Type = enum(LOW = 1, MID = 2 , HIGH = 3)

### Battery Class
* get_cycle : return current battery cycle.
* get_charge: return charge level .
* get_status: return: charging status: 1 for charging and 2 for discharging.
* charge_battery: get the minute, and this parameter times the charging rate will be the charging amount to be add to the current level.
* discharge_battery: 
        get the minutes: this parameter times the discharging rate will be the charging amount to be subtracted from the current level charge the battery. This way we represent the discharging for different user type.
        Factory Failure: considering the battery type anomaly rate, battery may crash.
* __date_cycle__: while adding a cycle to battery's life, will check Depreciation factor


In [5]:
class Battery():
    # to assign a unique id to each 
    battery_id = itertools.count(23)
    # construct a battery instance
    def __init__(self, bat_type, discharge_rate, charge_rate, level, anomaly_rate, health_cycle, charge_status, Inc, Dec):
        self.id = next (self.battery_id)
        self.type = bat_type
        self.discharge_rate = discharge_rate
        self.charge_rate = charge_rate
        self.charge_level = level
        self.anomaly_rate = anomaly_rate
        self.health_cycle = health_cycle
        self.charge_status = charge_status
        self.cycle = 1
        self.inc = Inc
        self.dec = Dec
    
    def get_cycle(self):
        '''
        @return: current battery cycle
        '''
        return self.cycle
     
    def get_charge(self):
        '''
        @return: charge level
        '''
        return round(self.charge_level)
    
    def get_status(self):
        '''
        @return: charging status: 1 for charging and 2 for discharging
        '''
        return self.charge_status
            
    def charge_battery(self, minutes):
        '''
        @minutes: this parameter times the charging rate will be the charging amount to be add to the current level
        charge the battery
        
        '''
        if self.charge_status == Charge_Stat.DISCHARGING:
            self.charge_status = Charge_Stat.CHARGING      
        
        increase_pc = self.charge_rate * minutes 
        self.charge_level += increase_pc
        if self.charge_level > 100: 
            self.charge_level = 100  
        self.charge_level = round(self.charge_level,2)

    def discharge_battery(self, minutes):
        '''
        @minutes: this parameter times the discharging rate will be the charging amount to be subtracted from the current level
        charge the battery. This way we represent the discharging for different user type.
        Factory Failure: considering the battery type anomaly rate, battery may crash.
        
        '''
        if self.charge_status == Charge_Stat.CHARGING:
            self.__update_cycle__()
        decrease_pc = self.discharge_rate * minutes 
        self.charge_level -= decrease_pc        
        if self.charge_level < 0: 
            self.charge_level = 0 
        self.charge_level = round(self.charge_level,2) 

        
    def __update_cycle__(self):
        '''
        while adding a cycle to battery's life, will check Depreciation factor
        '''
        self.charge_status = Charge_Stat.DISCHARGING    
        self.cycle +=1
        if (randrange(10000)<=self.anomaly_rate*10000):
            self.discharge_rate *= 5 * self.dec
            self.charge_rate /= self.inc *4               
        elif self.cycle % self.health_cycle == 0:
            self.discharge_rate *= self.dec
            self.charge_rate /= self.inc
                
                
    def info(self):
        print('Battery Info Id:{}\t Type: {} \n Cycle: {}\t Charge: {}\t Status: {}' \
                .format(self.id, self.type, self.get_cycle(), self.get_charge(), self.get_status()))
       

### Now let's check the battery class behavoiur by creating an instance and calling the charge and discharge methods

In [6]:
INC = 1.1
DEC = 1.1
bat1 = Battery(bat_type='bat_test', \
               discharge_rate= 2, \
               charge_rate = 10, \
               level = 40, \
               anomaly_rate = 0.05, \
               health_cycle = 5, \
               charge_status =Charge_Stat.CHARGING , Inc = INC , Dec = DEC)

In [7]:
bat1.info()
bat1.discharge_battery(2)
bat1.info()
bat1.charge_battery(3)
bat1.info()
bat1.discharge_battery(20)
bat1.info()

Battery Info Id:23	 Type: bat_test 
 Cycle: 1	 Charge: 40	 Status: 1
Battery Info Id:23	 Type: bat_test 
 Cycle: 2	 Charge: 18	 Status: 2
Battery Info Id:23	 Type: bat_test 
 Cycle: 2	 Charge: 25	 Status: 1
Battery Info Id:23	 Type: bat_test 
 Cycle: 3	 Charge: 0	 Status: 2


### Device Class:
This class represent a cell phone. It has a battery and can show the sensor anomaly behaviour by sending an invalid data while sending the charging level.
* charge: call battery charge method.
* discharge: call battery discharge method
* get_charge: return the battery charge level
* get_cycle: return battery cycle.


In [8]:
class Device():
    '''
    This class represent a cell phone. It has a battery and can show the sensor anomaly behaviour by sending an invalid data while sending the charging level.
    '''
    device_id = itertools.count(100)
    def __init__(self, model_name, brand_name, anomaly_rate, installed_battery):
        self.id = next(self.device_id)
        self.anomaly_rate = anomaly_rate
        self.model_name = model_name
        self.brand_name = brand_name
        self.battery = installed_battery
        
    def charge(self):
        '''
        call battery charge method.

        '''
        self.battery.charge_battery(1)
    
    def discharge(self,i):
        '''
        call battery discharge method

        '''
        self.battery.discharge_battery(i)
    
    def get_charge(self):
        '''
       return the battery charge level
        '''
        ch = self.battery.get_charge()
#         generating device anomaly
        ran = randrange(10000)
        if (ran <= self.anomaly_rate):
            if (randrange(100)<50):
                ch = randrange(50)
            else:
                ch = randrange(50,100)
        return ch

    def get_cycle(self):
        '''
        @return: battery cycle.

        '''
        return self.battery.get_cycle()
    
    def get_status(self):
        '''
        @return: battery status of charging or discharging.

        '''
        return self.battery.get_status()
    
    def is_charging(self):
        '''
        @ return: True if battery is in charging status (means that it was charge in the prebios change)
        '''
        return self.get_status() == Charge_Stat.CHARGING
    
    def info(self):
        print('Device Info Id:{}\t Brand: {}\t Model:{} \n Cycle: {}\t Charge: {}\t Status: {}' \
                .format(self.id, self.brand_name,self.model_name, self.get_cycle(), self.get_charge(), self.get_status()))        
        

### Now let's test the device class

In [9]:
dev1 = Device( 'S6 Plus', 'SAMSUNG', 0.05,bat1)

In [10]:
dev1.info()

Device Info Id:100	 Brand: SAMSUNG	 Model:S6 Plus 
 Cycle: 3	 Charge: 0	 Status: 2


### User Class
* get_user_group: return the user usage as text.
* __charge_battery__: increase battery charge level
* __discharge_battery__: decrease battery charge level
*  change_battery_level: based on the rule call battery_charge or discharge function
* export_json: This method create a json object of the user information and return

In [11]:
class User(): 
    user_id = itertools.count(0)
    def __init__(self, n, loc, lat, lang, u_type, device, consupmtion): 
        self.u_id = next(self.user_id) 
        self.full_name = n +str(self.u_id).zfill(6)
        self.location = loc 
        self.latitude = lat 
        self.langitude = lang 
        self.group = u_type 
        self.device = device  
        self.dateTime = ''
        self.user_type_consumption = consupmtion

    def info(self):  
        print ('User \n ID: {}\t User Name: {}\t Group: {} \t Consumption '.format(self.u_id, self.full_name, self.get_user_group(),self.user_type_consumption))
        print ('Location: \n Namw: {}\t Latitude: {}\t Langitude: {}'.format(self.location, self.latitude, self.langitude))
        print ('Deivce:\n ID: {} \t Brand: {}\t Model'.format(self.device.id , self.device.brand_name, self.device.model_name))
        print('Battery:\n ID : {} \t Type: {} \t Charge: {}\t Cycle: {}\t is Charging:{}'
              .format(self.device.battery.id, self.device.battery.type,self.device.get_charge(),
                self.device.get_cycle(),self.device.is_charging()))

  
    def get_user_group(self):
        '''
        @return: user usage as text.
        '''
        if self.group == User_Type.HIGH:
            return 'High'
        elif self.group == User_Type.MID:
            return 'Mid'
        else:
            return 'Low'
        
    def __charge_battery__(self): 
        '''
        increase battery charge level
        '''
        self.device.charge()

    def __discharge_battery__(self): 
        '''
        decrease battery charge level
        '''
        self.device.discharge(self.user_type_consumption)

    def change_battery_level(self): 
        '''
        based on the rule call battery_charge or discharge function
        '''
        if(( self.device.is_charging() and self.device.get_charge() < randrange(70,101) )
           or self.device.get_charge() <= randrange(0,30)): 
            self.__charge_battery__() 
        else: 
            self.__discharge_battery__() 
        return self.device.get_charge() 
    
    def set_date_time(self,dt):
        self.dateTime = dt
    
    def export_json(self): 
        '''
        This method create a json object of the user information and return
        '''
        data={} 
        data['u_id'] = self.u_id         
        data['full_name'] = self.full_name 
        data['Location'] = self.location 
        data['Latitude'] = self.latitude 
        data['Langitude'] = self.langitude 
        data['User_Type'] = self.group 
        data['Battery_Type'] = self.device.battery.type
        data['Battery_Level'] = self.device.get_charge()
        data['Battery_Status'] = self.device.get_status()
        data['Battery_Cycle_No'] = self.device.get_cycle()
        data['DateTime'] = self.dateTime
        return data 
    
    def export_string_json(self):
       
        jdata = self.export_json()
        json_string = json.dumps(jdata) 
        return json_string

    

### Let's test our User class and create an instance

In [12]:
consupmtion =8 
u_test =  User('ali','L1',1 ,2 ,User_Type.HIGH, dev1  , consupmtion )  
u_test.info()
print (u_test.export_json())
for u in range(10):
    u_test.change_battery_level()
    print('chargre:{}\t cycle:{}\t status:{}'.format(u_test.device.get_charge(),u_test.device.get_cycle(),u_test.device.get_status()))

User 
 ID: 0	 User Name: ali000000	 Group: High 	 Consumption 
Location: 
 Namw: L1	 Latitude: 1	 Langitude: 2
Deivce:
 ID: 100 	 Brand: SAMSUNG	 Model
Battery:
 ID : 23 	 Type: bat_test 	 Charge: 0	 Cycle: 3	 is Charging:False
{'u_id': 0, 'full_name': 'ali000000', 'Location': 'L1', 'Latitude': 1, 'Langitude': 2, 'User_Type': 3, 'Battery_Type': 'bat_test', 'Battery_Level': 0, 'Battery_Status': 2, 'Battery_Cycle_No': 3, 'DateTime': ''}
chargre:2	 cycle:3	 status:1
chargre:5	 cycle:3	 status:1
chargre:7	 cycle:3	 status:1
chargre:9	 cycle:3	 status:1
chargre:11	 cycle:3	 status:1
chargre:14	 cycle:3	 status:1
chargre:16	 cycle:3	 status:1
chargre:18	 cycle:3	 status:1
chargre:20	 cycle:3	 status:1
chargre:23	 cycle:3	 status:1


### This is the main class which use all other classes to generate the dataset
* load_config:  the code will load dataset and environment settings from the config.ini.
* dataset_load:  the code load the basic csv files from the data folder.
* print_dataset_head:  the code will print all the datasets' first N (input) rows to check if loaded  properly
* print_parameter: print the loaded parameters from the config.ini
*  __create_battery__:
    @param bat: an instance of a battery object
    create a battery
    random selecting a battery type from the given list (battery.csv) file
    battery_no is from config.ini is the max number of batteries
* __create_device__:
    @param devicbate: an instance of a battery object
    create a device with a battery
    randomly select a device type from the given list (device.csv) file.
    device_no is from config.ini is the max number of devices
* __create_user_:
    @param u: user id
    @param device: an instance of a device object
    randomly selecting the user type and user location from the given csv file and in the range of the max no of locations and user types.
* __generate_user_device:
    In this method for the number of requested users (read from the cofig.ini) the method will create a battery instance then 
    will create a device and assign the battery to it. Finally, it will create a user.
    Finally, it will save a list of created users to a csv file (name of the file from config.ini file)
* generate_main_dataset:
    This method controls whole process:
    •	Create list of users. Each user has a device and each device (form device type csv file) has a battery (form battery type csv file)
    •	Generate the list of sending data from the devices based on the start and dates and given interval time in the config.ini
    •	Creating a csv dataset including a record for each user and each time interval 
    •	Save the generated list in csv format in the Data_ge folde

    


In [13]:
class Data_generator():
    def __init__(self, config_filename='config.ini'):   
        self.obj_stat= True
        self.load_config(config_filename=config_filename)
        self.dataset_load()
    #         self.print_dataset_head(5)
      
 
    def load_config(self,config_filename):
        '''
        the code will load dataset and environment settings from the config.ini
        ''' 
        if (self.obj_stat== False):
            print('Error while initialization')
            return
        print('loading the config file')
        self.config = configparser.ConfigParser()
        self.config.read(config_filename)
        #load generation setting
        self.user_no = int( self.config['SETTING']['USER_NO'])
        self.user_type_no = int(self.config['SETTING']['USER_TYPE_NO'])
        self.location_no = int(self.config['SETTING']['LOCATION_NO'])
        self.battery_no = int(self.config['SETTING']['BATTARY_NO'])
        self.device_no = int(self.config['SETTING']['DEVICE_NO'])
        self.data_no = int(self.config['SETTING']['DATA_BO_NO']      )
        self.start_year = int(self.config['SETTING']['START_YEAR'])
        self.start_month = int(month_converter(self.config['SETTING']['START_MONTH']))
        self.start_day = int(self.config['SETTING']['START_DAY'])
        self.end_year = int(self.config['SETTING']['END_YEAR'])
        self.end_month = int(month_converter(self.config['SETTING']['END_MONTH']))    
        self.end_day = int(self.config['SETTING']['START_DAY'])        
        self.interval = int(self.config['SETTING']['INTERVAL'])
        self.inc_rate = float(self.config['SETTING']['INC_RATE'])
        self.dec_rate = float(self.config['SETTING']['DEC_RATE'])
        self.Low_usage = int(self.config['SETTING']['LOW_USAGE'])
        self.Mid_usage = int(self.config['SETTING']['MID_USAGE'])
        self.High_usage = int(self.config['SETTING']['HIGH_USAGE'])
        
        #load basic datasets name
        
        self.data_folder = self.config['DATASET']['DATA_FOLDER']
        self.data_folder_ge = self.config['DATASET']['DATA_FOLDER_GE']        
        self.userType_ds_path = str(self.data_folder)  + os.sep + str(self.config['DATASET']['USER_TYPE_DS'])
        self.location_ds_path = str(self.data_folder)  + os.sep + str(self.config['DATASET']['LOCATION_DS'])
        self.battery_ds_path = str(self.data_folder)  + os.sep + str(self.config['DATASET']['BATTERY_DS'])
        self.device_ds_path = str(self.data_folder)  + os.sep + str(self.config['DATASET']['DEVICE_DS'])
        self.user_device_ds_path = str(self.data_folder_ge)  + os.sep + str(self.config['DATASET']['USER_DEVICE'])
        
      
    def dataset_load(self):
        '''
        the code load the basic csv files from the data folder to the app
        ''' 
        if (self.obj_stat== False):
            print('Error while initialization')
            return
        print('Loading the init datasets:')
        self.locations = pd.read_csv(self.location_ds_path)
        print('Location dataset is loaded from '+ self.location_ds_path)
        self.battery = pd.read_csv(self.battery_ds_path)
        print('Battery dataset is loaded from '+ self.battery_ds_path)
        self.user_types = pd.read_csv(self.userType_ds_path)
        print('UserTypes dataset is loaded from '+ self.userType_ds_path)
        self.device = pd.read_csv(self.device_ds_path)
        print('Device dataset is loaded from '+ self.device_ds_path)

        
    def print_dataset_head(self, N):
        '''
        @param N: number of head rows to print
        the code will print all the datasets' first N rows to check if loaded  properly
        '''
        if (self.obj_stat== False):
            print('Error while initialization')
            return
        print('\n\nSample Users types\n',self.user_types.head(N))
        print('\n\nSample locations\n',self.locations.head(N))
        print('\n\nSample batteries types\n',self.battery.head(N))
        print('\n\nSample devices types\n',self.device.head(N))


    def __datetime_range__(self, start, end, delta):
        current = start
        while current < end:
            yield current
            current += delta
                

    def print_parameter(self):
        '''
        print loaded parameters    
        '''
        if (self.obj_stat== False):
            print('Error while initialization')
            return
        print('\n User No:\t',self.user_no)
        print('\n User Type No:\t',self.user_type_no)
        print('\n Location No:\t',self.location_no)
        print('\n Battery No:\t',self.battery_no)
        print('\n Device No:\t',self.device_no)
        print('\n Data No:\t',self.data_no)
        print('\n From No:\t',self.year ,' ', self.start)
        print('\n To   No:\t',self.year,' ', self.end)
        print('\n Interval:\t',self.interval,'mins')
    

    def __time_priod__(self):
        '''
        Generate the list of sending data from the devices based on the start and dates and given interval time in the config.ini
        '''
        start_date = datetime(self.start_year, self.start_month, self.start_day, random.randint(0, 23))
        end_date = datetime(self.end_year, self.end_month, self.end_day, random.randint(0, 23))
#         mins=random.randint(1, self.interval+1)
        interval = self.interval
        delta = timedelta(minutes = interval)
        self.period = self.__datetime_range__(start_date, end_date, delta)
        print('Start Date:\t{},\n End Date:\t{},\n Interval:\t{}'.format(start_date, end_date, delta))
        

    def __create_battery__(self):
        '''
        @param bat: an instance of a battery object
        create a battery
        random selecting a battery type from the given list (battery.csv) file
        battery_no is from config.ini is the max number of batteries
        '''
        r_bat = randrange(self.battery_no)
        bat = Battery(
                                self.battery.iloc[r_bat]['type'], 
                                round((100 / (int(self.battery.iloc[r_bat]['drain']) * 24 *60 /self.interval)),2), 
                                round(100/ (int(self.battery.iloc[r_bat]['recharge'])/self.interval)), 
                                randrange(20, 60 ),
                                self.battery.iloc[r_bat]['anomaly_rate'], 
                                self.battery.iloc[r_bat]['health_cycle'],
                                Charge_Stat.DISCHARGING,
                                self.inc_rate,
                                self.dec_rate)
        return bat

    def __create_device__(self, bat):
        '''
        @param devicbate: an instance of a battery object
        create a device with a battery
        randomly select a device type from the given list (device.csv) file.
        device_no is from config.ini is the max number of devices
        '''
        r_device = randrange(self.device_no)
        dev = Device(self.device.iloc[r_device]['model_name'], \
                                self.device.iloc[r_device]['brand_name'], \
                                self.device.iloc[r_device]['anomaly_rate'], \
                                bat)
        return dev

    def __create_user__(self, u, device):
        '''
        @param u: user id
        @param device: an instance of a device object
        randomly selecting the user type and user location from the given csv file and in the range of the max no of locations and user types 
        ''' 
        r_loc = randrange(self.location_no)
        r_user_type = randrange(self.user_type_no) + 1 
        if (r_user_type == User_Type.LOW):
            user_consumption = self.Low_usage
        elif (r_user_type == User_Type.MID): 
            user_consumption = self.Mid_usage
        elif (r_user_type == User_Type.HIGH): 
            user_consumption = self.High_usage

        usr = User (    'user',\
                                self.locations.iloc[r_loc]['name'], \
                                self.locations.iloc[r_loc]['lat'], \
                                self.locations.iloc[r_loc]['lang'], \
                                r_user_type, \
                                device, \
                                user_consumption)
        return usr
    

    def __generate_user_device__(self):
        '''
        @param self: Data_generator instance
        In this method for the number of requested users (read from the cofig.ini) the method will create a battery instance then 
        will create a device and assign the battery to it. Finally, it will create a user.
        Finally, it will save a list of created users to a csv file (name of the file from config.ini file)
        '''
        print('Assigning Devices to Users')
        self.user_list=[]
        print ('mapping {} users to a random device and battery type'.format(self.user_no))
        for u in range(self.user_no):
            battery = self.__create_battery__()
            device = self.__create_device__(battery)               
            usr = self.__create_user__(u, device)
            self.user_list.append(usr)
        append_json_to_csv( [x.export_json() for x in self.user_list],self.data_folder_ge, self.user_device_ds_path)
        print ('Save CSV: the list of users and their details saved in the path :',self.user_device_ds_path)
        
    def print_generated_df_head(self, N=2, info_flag=False, head_flag=True):
        '''
        @param N: number of rows to print in the dataframe.head(N) commnad  => default = 2
        @param info_flag: True causes to print the dataframe.info() command => default = False
        @param head_flag: True causes to print the dataframe.head() command => default = True
        '''
        
        if (self.obj_stat== False):
            print('Error while initialization')
            return          
        battery_df=[]
        for bt in range(self.battery_no):
            if  not( os.path.isfile(self.location_ds_path)):
                    print ('file reading Error')
                    return
            path = self.data_folder_ge +os.sep+'final-bat' + str(bt+1) + '.csv'
            print('\n***********************************************************************************************************')
            print('*************************** Print Info and Head of dataframe of battery type {}****************************'.format(str(bt+1)))
            print('Loading data from ' +path)
            battery_df.append( pd.read_csv(path))
            if (info_flag):
                print(battery_df[bt].info())
            if (head_flag):
                print(battery_df[bt].head(N))


    def generate_main_dataset(self):
        '''
        This method controls whole process:
        •	Create list of users. Each user has a device and each device (form device type csv file) has a battery (form battery type csv file)
        •	Generate the list of sending data from the devices based on the start and dates and given interval time in the config.ini
        •	Creating a csv dataset including a record for each user and each time interval 
        •	Save the generated list in csv format in the Data_ge folde

        '''
        if (self.obj_stat== False):
            print('Error while initialization')
            return
        # Create list of users. Each user has a device and each device (form device type csv file) has a battery (form battery type csv file)
        self.__generate_user_device__() 
        # Generate the list of sending data from the devices based on the start and dates and given interval time in the config.ini
        self.__time_priod__()
        dts = [dt.strftime('%m-%d-%Y %H:%M:%S') for dt in self.period]
        print ('Number of iterations data sent from each device: ',len(dts))
        records = [[] for _ in range(self.battery_no)]
        for d_t in dts:
            for u in self.user_list:
                u.change_battery_level()
                u.set_date_time(d_t)
                bt = int(u.device.battery.type.strip('bat'))-1
                records[bt].append(u.export_json())
                                     
        for bt in range(self.battery_no):
            path = self.data_folder_ge +os.sep+'final-bat' + str(bt+1) + '.csv'
            append_json_to_csv(records[bt], self.data_folder_ge,path)
            


## before start running the code first set the config_filename var 

### The following code will initialize a data generator object

In [16]:
config_filename = 'config.ini'
dg = Data_generator(config_filename)
dg.generate_main_dataset()

loading the config file
Loading the init datasets:
Location dataset is loaded from data2\location.csv
Battery dataset is loaded from data2\battery.csv
UserTypes dataset is loaded from data2\user_types.csv
Device dataset is loaded from data2\device.csv
Assigning Devices to Users
mapping 3000 users to a random device and battery type

 file data2_ge\user_list.csv saving process started with the header 
['Battery_Cycle_No', 'Location', 'Battery_Status', 'Battery_Level', 'Battery_Type', 'u_id', 'DateTime', 'full_name', 'Latitude', 'Langitude', 'User_Type']

 
 file data2_ge\user_list.csv saving process finished
Save CSV: the list of users and their details saved in the path : data2_ge\user_list.csv
Start Date:	2018-01-01 03:00:00,
 End Date:	2019-01-01 20:00:00,
 Interval:	1:00:00
Number of iterations data sent from each device:  8777

 file data2_ge\final-bat1.csv saving process started with the header 
['Battery_Cycle_No', 'Location', 'Battery_Status', 'Battery_Level', 'Battery_Type', 'u

### Final Step to generate the data based on the setting in the config.ini

In [17]:
dg.print_generated_df_head(5,info_flag =True,head_flag = True)

loading the config file

***********************************************************************************************************
*************************** Print Info and Head of dataframe of battery type 1****************************
Loading data from data2_ge\final-bat1.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8627791 entries, 0 to 8627790
Data columns (total 11 columns):
Battery_Cycle_No    int64
Location            object
Battery_Status      int64
Battery_Level       int64
Battery_Type        object
u_id                int64
DateTime            object
full_name           object
Latitude            float64
Langitude           float64
User_Type           int64
dtypes: float64(2), int64(5), object(4)
memory usage: 724.1+ MB
None
   Battery_Cycle_No Location  Battery_Status  Battery_Level Battery_Type  \
0                 1      SFU               2             55         bat1   
1                 1      SFU               2             17         bat1   
2             